In [1]:
import numpy as np
import pandas as pd
import math
import random

In [2]:
demand = np.concatenate(([0],np.repeat(1, 10)))
distance = [[0, 4.12, 2.24, 5, 4.24, 2.24, 5.1, 5.1, 3.16, 3.61, 6.08],
            [4.12, 0, 3.16, 5.66, 7.28, 6, 9, 9.22, 6.4, 3.16, 2.83],
            [2.24, 3.16, 0, 3.16, 4.12, 3.16, 6.08, 6.71, 5.39, 4.47, 5.83],
            [5, 5.66, 3.16, 0, 3.61, 4.47, 6.4, 7.81, 8.06, 7.62, 8.49],
            [4.24, 7.28, 4.12, 3.61, 0, 2.24, 2.83, 4.47, 6.32, 7.81, 9.85],
            [2.24, 6, 3.16, 4.47, 2.24, 0, 3, 3.61, 4.12, 5.83, 8.25],
            [5.1, 9, 6.08, 6.4, 2.83, 3, 0, 2, 5.66, 8.54, 11.18],
            [5.1, 9.22, 6.71, 7.81, 4.47, 3.61, 2, 0, 4.47, 8.06, 11],
            [3.16, 6.4, 5.39, 8.06, 6.32, 4.12, 5.66, 4.47, 0, 4.12, 7.28],
            [3.61, 3.16, 4.47, 7.62, 7.81, 5.83, 8.54, 8.06, 4.12, 0, 3.16],
            [6.08, 2.83, 5.83, 8.49, 9.85, 8.25, 11.18, 11, 7.28, 3.16, 0]]

#cartesian coordinate
ccoor = [[0, 0],[4, 1],[1, 2],[0, 5],[-3, 3],[-2, 1],[-5, 1],[-5, -1],[-1, -3],[3, -2],[6, -1]]

nodes = len(distance) #number of nodes, inluded depot
tours = nodes-1 # number of identical vehicles/tours, upper bound equals to the number of customers
capacity = 4 #capacity of each vehicle
timeRes = 16 #working time window of each vehicle

In [3]:
def t_demand(tour):
    d = 0
    for nd in tour[1:len(tour)-1]:
        d += demand[nd]
    return d

def t_time(tour):
    tc=0
    for i in range(len(tour)-1): #for each node in solution except last node
        tc += distance[tour[i]][tour[i+1]]
    return tc

In [4]:
def totalcost(solution):
    tc = 0
    for tour in solution:
        tc += t_time(tour)
    
    return tc

In [5]:
#solution encode: SET of tours => do not consider the index of tours => break symmetry: {tour1, tour2} = {tour2, tour1}
solution = set()

#starting solution = delivery to one customer per tour
# no split delivery
for i in range(1,nodes):
    solution.add((0,i,0))
solution.add((0,0))
#    tour = (0,i,0)
#    solution.add( (tour, capacity-t_demand(tour), timeRes-t_time(tour))  ) #(tour, demand slack, time slack)
#solution.add( ((0,0),capacity,timeRes) ) #represent the chance to add new tour

In [6]:
solution

{(0, 0),
 (0, 1, 0),
 (0, 2, 0),
 (0, 3, 0),
 (0, 4, 0),
 (0, 5, 0),
 (0, 6, 0),
 (0, 7, 0),
 (0, 8, 0),
 (0, 9, 0),
 (0, 10, 0)}

In [7]:
def best_insert(tour, node):
    length_change = np.zeros(len(tour) - 1) #list of change in length for each possible position to insert new node

    for position in range(len(tour) - 1): #iterate through possible positions in current tour
        temp_tour = tour[: position+1] + (node,) + tour[position+1 :]

        #calculate the change in length between current tour and temp tour
        pre_ins_node = tour[position]
        nex_ins_node = tour[position + 1]
        if pre_ins_node == nex_ins_node:
            length_change[position] = distance[pre_ins_node][node] + distance[node][nex_ins_node]
        else:
            length_change[position] = distance[pre_ins_node][node] + distance[node][nex_ins_node] - distance[pre_ins_node][nex_ins_node]

    min_position = np.argmin(length_change) #get the position which create the shortest length change
    #length += length_change[min_position]
    new_tour = tour[: min_position+1] + (node,) + tour[min_position+1 :]
    
    return (new_tour, - length_change[min_position])

In [8]:
# a move = move a node i from tour k1 to tour k2
def move(tour1, tour2, ind):
    new_t1 = tour1[:ind] + tour1[ind+1:] #remove node with index ind from tour 1
    (new_t2, change) = best_insert(tour2, tour1[ind]) #insert new node into tour 2
    
    delta = distance[tour1[ind-1]][tour1[ind]] + distance[tour1[ind]][tour1[ind+1]] - distance[tour1[ind-1]][tour1[ind+1]] + change
    
    return (new_t1, new_t2, delta) #delta = cost of old solution - cost of new solution

In [9]:
# a set of neighborhood N(S) = {solutions obtained by applying a single local transformation (move, i.e. add or drop) to S}
# new solution must be feasible

def neighborhood(solution):  
    
    #random select a node:
    nd = np.random.randint(1,nodes)
    
    #find the current tour of the random node
    for tour in solution:
        if nd in tour:
            ind = tour.index(nd)
            tour1 = tour
            break    
    # find feasible tour set to swap
    possible_tours = []
    
    #demand constraint
    for tour in solution.difference([tour1]):
        if (capacity - t_demand(tour) >= demand[nd]): #demand constraint
            possible_tours.append(tour) #always include at least tour (0,0)
    
    #time constraint
    feasible = False
    while not feasible:
        tour2 = random.choice(possible_tours)
        if (timeRes - t_time(tour2)) >= (- best_insert(tour2, nd)[1]): #always find at least tour (0,0)
            feasible = True
            break
        else: possible_tours.remove(tour2)
    
    (new_tour1, new_tour2, delta) = move(tour1, tour2, ind)
    temp = solution.difference([tour1, tour2]).union([new_tour1, new_tour2,(0,0)])
    
    return (temp, delta) #, new_tour1, new_tour2,tour2,tour1)

In [10]:
#main function
def SA (solution, cost, initial=1, cool_factor=0.95, eval_times=10000, 
        TimeLimit = None, non_improvement = None, max_iteration = None, min_temp = None):

    best = cost
    best_solution = solution 
    best_iter = 0
    temperature = initial*best
    print(solution)
    print(best)
    
    #time limit condition
    from time import process_time 
    start_time = process_time()
     
    #max iteration condition
    iteration = 0
    non_iter = 0
    
    terminated = False
    while not terminated: #termination condition 
        
        improvement = False
        iteration += 1 
        
        #evaluation at given alpha
        for k in range(eval_times):
            (next_solution, delta) = neighborhood(solution)
            #print(next_solution, totalcost(next_solution))
            
            if delta > 0 or (np.random.random() < math.exp(delta/temperature)): 
                #print('****', cost, delta)
                cost = cost - delta
                solution = next_solution
            if cost < best:
                best = cost
                best_solution = solution
                best_iter = iteration
                improvement = True
                print(best, '   ', temperature)
                
        temperature = cool_factor*temperature

        if (TimeLimit is not None): #time elapsed termination
            runtime = process_time() - start_time
            if (runtime >= TimeLimit): 
                print('Reach TimeLimit')
                terminated = True
                
        elif (non_improvement is not None): #max iteration without new best solution
            if improvement: 
                non_iter = 0
            else: non_iter += 1
            if non_iter >= non_improvement:
                print('Reach max number of iteration without new best solution')
                terminated = True
        
        elif (max_iteration is not None):
            if iteration >= max_iteration:
                print('Reach max number of iteration')
                terminated = True
                
        elif (min_temp is not None):
            if  temperature < min_temp : 
                print('Reach min temperature')
                terminated = True
                
        else: 
            if  temperature < (0.95**100) * initial*length: #default option 
                print('Reach default min temperature')
                terminated = True
    
    print("Terminate")
    print(best_solution)
    print(best)

In [11]:
SA (solution, totalcost(solution), initial=1, cool_factor=0.95, eval_times=100, TimeLimit = 60)

{(0, 2, 0), (0, 9, 0), (0, 0), (0, 7, 0), (0, 3, 0), (0, 10, 0), (0, 4, 0), (0, 5, 0), (0, 1, 0), (0, 8, 0), (0, 6, 0)}
81.78000000000002


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: overflow encountered in double_scalars


Reach TimeLimit
Terminate
{(0, 9, 10, 1, 0), (0, 4, 3, 2, 0), (0, 0), (0, 5, 6, 7, 8, 0)}
41.83999999999985


In [12]:
if 9 in (8,9):
    
    print('OK')

OK


In [13]:
(9,8,7,5)[0:2] + (1,) + (6,4)[0:1]

(9, 8, 1, 6)

In [14]:
best_insert((0,0), (0,9,0)[1])

((0, 9, 0), -7.22)